# Ways to test `@skip_if_unavailable`

This are some “internal” testing notes for `@skip_if_unavailable`.

If you're looking for information on how to *use* the `@skip_if_unavailable`
decorator, then this notebook is not what you want. Instead:

- See [`README.md`](../../README.md).

- See also the code of `@subaudit.skip_if_unavailable` (it's simple).

- If those are insufficient, see the “`@subaudit.skip_if_unavailable`” section
  of [`from_readme.ipynb`](from_readme.ipynb), which uses techniques from here
  to simulate the Python 3.7 situation on later versions of Python, but which
  ties it into the example code in [`README.md`](../../README.md).

The unit tests (in [`tests/`](../../tests/)) did not end up use the technique
shown here of patching `sys.version_info` and reloading the `subaudit` module.
Note that this was considered for the tests *of* `@skip_if_unavailable`—it is
*definitely not* something that would be done as part of *actually using*
`@skip_if_unavailable` in a test suite.

In [1]:
import contextlib
import importlib
import sys

import unittest.mock

import subaudit

In [2]:
sys.modules['sysaudit'] = sys

In [3]:
def demo_skip_if_unavailable(*, patch_version):
    if patch_version is None:
        maybe_patcher = contextlib.nullcontext()
    else:
        maybe_patcher = unittest.mock.patch('sys.version_info', patch_version)

    with maybe_patcher:
        importlib.reload(subaudit)
        mock_test = unittest.mock.Mock(wraps=lambda: None)
        subaudit.skip_if_unavailable(mock_test)()
        return mock_test.mock_calls

In [4]:
demo_skip_if_unavailable(patch_version=None)

[call()]

In [5]:
demo_skip_if_unavailable(patch_version=(3, 7))

SkipTest: Python Runtime Audit Hooks (PEP 578) were introduced in Python 3.8.